<a href="https://colab.research.google.com/github/JcorredorG/Activity-6-POO/blob/main/8_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# ============================================
# NÓMINA CON INTERFAZ EN GOOGLE COLAB
# ============================================

# (Solo la PRIMERA vez en un notebook nuevo, descomenta estas 3 líneas,
# ejecútalas, y luego vuelve a comentar o ignora)
# !pip install ipywidgets
# from google.colab import output
# output.enable_custom_widget_manager()

from enum import Enum
from dataclasses import dataclass
from typing import List
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd


# ==============================
# 1. MODELO DE DATOS
# ==============================

class Cargo(Enum):
    DIRECTIVO = "Directivo"
    ESTRATEGICO = "Estratégico"
    OPERATIVO = "Operativo"


class Genero(Enum):
    MASCULINO = "Masculino"
    FEMENINO = "Femenino"


@dataclass
class Empleado:
    nombre: str
    apellidos: str
    cargo: Cargo
    genero: Genero
    salario_dia: float
    dias_trabajados: int
    otros_ingresos: float
    pagos_salud: float
    aporte_pensiones: float

    @property
    def salario_mensual(self) -> float:
        """
        salario mensual = salario_dia * dias_trabajados
                          + otros_ingresos
                          - pagos_salud
                          - aporte_pensiones
        """
        return (
            self.salario_dia * self.dias_trabajados
            + self.otros_ingresos
            - self.pagos_salud
            - self.aporte_pensiones
        )


class Nomina:
    def __init__(self):
        self.empleados: List[Empleado] = []

    def agregar_empleado(self, empleado: Empleado):
        self.empleados.append(empleado)

    def calcular_total(self) -> float:
        return sum(e.salario_mensual for e in self.empleados)

    def a_dataframe(self) -> pd.DataFrame:
        datos = [
            {
                "Nombre": e.nombre,
                "Apellidos": e.apellidos,
                "Cargo": e.cargo.value,
                "Género": e.genero.value,
                "Salario día": e.salario_dia,
                "Días": e.dias_trabajados,
                "Otros ingresos": e.otros_ingresos,
                "Pagos salud": e.pagos_salud,
                "Pensiones": e.aporte_pensiones,
                "Salario mensual": e.salario_mensual,
            }
            for e in self.empleados
        ]
        if not datos:
            return pd.DataFrame(columns=[
                "Nombre", "Apellidos", "Cargo", "Género",
                "Salario día", "Días", "Otros ingresos",
                "Pagos salud", "Pensiones", "Salario mensual"
            ])
        return pd.DataFrame(datos)

    def guardar_en_archivo(self, ruta_archivo: str = "Nomina.txt") -> str:
        """
        Guarda los datos de todos los empleados y el total de la nómina
        en un archivo de texto. Devuelve la ruta absoluta del archivo.
        """
        p = Path(ruta_archivo).resolve()
        with p.open("w", encoding="utf-8") as f:
            for e in self.empleados:
                f.write(f"Nombre: {e.nombre}\n")
                f.write(f"Apellidos: {e.apellidos}\n")
                f.write(f"Cargo: {e.cargo.value}\n")
                f.write(f"Género: {e.genero.value}\n")
                f.write(f"Salario por día: {e.salario_dia:,.2f}\n")
                f.write(f"Días trabajados: {e.dias_trabajados}\n")
                f.write(f"Otros ingresos: {e.otros_ingresos:,.2f}\n")
                f.write(f"Pagos salud: {e.pagos_salud:,.2f}\n")
                f.write(f"Aportes pensiones: {e.aporte_pensiones:,.2f}\n")
                f.write(f"Salario mensual: {e.salario_mensual:,.2f}\n")
                f.write("-" * 40 + "\n")
            f.write(f"TOTAL NÓMINA: {self.calcular_total():,.2f}\n")
        return str(p)


# ==============================
# 2. INTERFAZ CON IPYWIDGETS
# ==============================

nomina = Nomina()

# ----- Campos de entrada -----
nombre_input = widgets.Text(
    description="Nombre:",
    placeholder="Nombre",
    layout=widgets.Layout(width="300px")
)

apellidos_input = widgets.Text(
    description="Apellidos:",
    placeholder="Apellidos",
    layout=widgets.Layout(width="300px")
)

cargo_input = widgets.Dropdown(
    options=[(c.value, c) for c in Cargo],
    value=Cargo.DIRECTIVO,
    description="Cargo:",
    layout=widgets.Layout(width="300px")
)

genero_input = widgets.Dropdown(
    options=[(g.value, g) for g in Genero],
    value=Genero.MASCULINO,
    description="Género:",
    layout=widgets.Layout(width="300px")
)

salario_dia_input = widgets.FloatText(
    description="Salario día:",
    value=0.0,
    layout=widgets.Layout(width="300px")
)

dias_input = widgets.BoundedIntText(
    description="Días:",
    value=1,
    min=1,
    max=31,
    layout=widgets.Layout(width="300px")
)

otros_ingresos_input = widgets.FloatText(
    description="Otros ingresos:",
    value=0.0,
    layout=widgets.Layout(width="300px")
)

pagos_salud_input = widgets.FloatText(
    description="Pagos salud:",
    value=0.0,
    layout=widgets.Layout(width="300px")
)

pensiones_input = widgets.FloatText(
    description="Pensiones:",
    value=0.0,
    layout=widgets.Layout(width="300px")
)

# ----- Botones -----
agregar_btn = widgets.Button(
    description="Agregar empleado",
    button_style="success"
)

borrar_btn = widgets.Button(
    description="Borrar campos",
    button_style="warning"
)

ver_nomina_btn = widgets.Button(
    description="Mostrar nómina",
    button_style="info"
)

guardar_btn = widgets.Button(
    description="Guardar en Nomina.txt",
    button_style=""
)

# ----- Áreas de salida -----
tabla_output = widgets.Output()
mensaje_output = widgets.Output()


# ==============================
# 3. FUNCIONES DE LA INTERFAZ
# ==============================

def limpiar_campos():
    nombre_input.value = ""
    apellidos_input.value = ""
    cargo_input.value = Cargo.DIRECTIVO
    genero_input.value = Genero.MASCULINO
    salario_dia_input.value = 0.0
    dias_input.value = 1
    otros_ingresos_input.value = 0.0
    pagos_salud_input.value = 0.0
    pensiones_input.value = 0.0


def on_agregar_clicked(_):
    with mensaje_output:
        clear_output()
        try:
            nombre = nombre_input.value.strip()
            apellidos = apellidos_input.value.strip()

            if not nombre or not apellidos:
                raise ValueError("Nombre y apellidos son obligatorios.")

            salario_dia = float(salario_dia_input.value)
            dias = int(dias_input.value)
            otros = float(otros_ingresos_input.value)
            salud = float(pagos_salud_input.value)
            pensiones = float(pensiones_input.value)

            if dias < 1 or dias > 31:
                raise ValueError("Los días trabajados deben estar entre 1 y 31.")

            empleado = Empleado(
                nombre=nombre,
                apellidos=apellidos,
                cargo=cargo_input.value,
                genero=genero_input.value,
                salario_dia=salario_dia,
                dias_trabajados=dias,
                otros_ingresos=otros,
                pagos_salud=salud,
                aporte_pensiones=pensiones
            )

            nomina.agregar_empleado(empleado)
            print(f"Empleado agregado: {nombre} {apellidos}")
        except ValueError as e:
            print(f"ERROR: {e}")


def on_borrar_clicked(_):
    with mensaje_output:
        clear_output()
        limpiar_campos()
        print("Campos borrados.")


def on_ver_nomina_clicked(_):
    with tabla_output:
        clear_output()
        df = nomina.a_dataframe()
        if df.empty:
            print("No hay empleados en la nómina.")
        else:
            display(df.style.format({
                "Salario día": "{:,.2f}",
                "Otros ingresos": "{:,.2f}",
                "Pagos salud": "{:,.2f}",
                "Pensiones": "{:,.2f}",
                "Salario mensual": "{:,.2f}",
            }))
            print(f"TOTAL NÓMINA MENSUAL: {nomina.calcular_total():,.2f}")


def on_guardar_clicked(_):
    with mensaje_output:
        clear_output()
        if not nomina.empleados:
            print("No hay empleados para guardar.")
            return
        ruta = nomina.guardar_en_archivo("Nomina.txt")
        print(f"Archivo Nomina.txt guardado en:\n{ruta}")


# Asociar eventos
agregar_btn.on_click(on_agregar_clicked)
borrar_btn.on_click(on_borrar_clicked)
ver_nomina_btn.on_click(on_ver_nomina_clicked)
guardar_btn.on_click(on_guardar_clicked)

# ==============================
# 4. MOSTRAR LA "VENTANA PRINCIPAL"
# ==============================

formulario = widgets.VBox([
    nombre_input,
    apellidos_input,
    cargo_input,
    genero_input,
    salario_dia_input,
    dias_input,
    otros_ingresos_input,
    pagos_salud_input,
    pensiones_input,
])

botones1 = widgets.HBox([agregar_btn, borrar_btn])
botones2 = widgets.HBox([ver_nomina_btn, guardar_btn])

ui = widgets.VBox([
    widgets.HTML("<h2>Nómina - Ejercicio 8.4</h2>"),
    formulario,
    botones1,
    botones2,
    widgets.HTML("<hr>"),
    tabla_output,
    mensaje_output,
])

display(ui)
